In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import round
from pyspark.sql import Window
from pyspark.sql.functions import col
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/26 14:45:44 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.178.12 instead (on interface en0)
22/09/26 14:45:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 14:45:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/26 14:45:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/26 14:45:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
final = spark.read.parquet('../data/curated/merchant_info.parquet')

In [3]:
max_Take_rate = float(final.agg({"Take_rate": "max"}).collect()[0][0])
min_Take_rate = float(final.agg({"Take_rate": "min"}).collect()[0][0])

max_Avg_amount_monthly= float(final.agg({"Avg_amount_monthly": "max"}).collect()[0][0])
min_Avg_amount_monthly = float(final.agg({"Avg_amount_monthly": "min"}).collect()[0][0])

max_Avg_count_monthly= float(final.agg({"Avg_count_monthly": "max"}).collect()[0][0])
min_Avg_count_monthly = float(final.agg({"Avg_count_monthly": "min"}).collect()[0][0])

max_Order_avg_value = float(final.agg({"Order_avg_value": "max"}).collect()[0][0])
min_Order_avg_value = float(final.agg({"Order_avg_value": "min"}).collect()[0][0])

max_avg_prob_fraud_cus = float(final.agg({"avg_prob_fraud_cus": "max"}).collect()[0][0])
min_avg_prob_fraud_cus = float(final.agg({"avg_prob_fraud_cus": "min"}).collect()[0][0])

max_num_of_fraud = float(final.agg({"num_of_fraud": "max"}).collect()[0][0])
min_num_of_fraud = float(final.agg({"num_of_fraud": "min"}).collect()[0][0])

max_count_cus_per_mon = float(final.agg({"count_cus_per_mon": "max"}).collect()[0][0])
min_count_cus_per_mon = float(final.agg({"count_cus_per_mon": "min"}).collect()[0][0])

max_fix_cus_num = float(final.agg({"fix_cus_num": "max"}).collect()[0][0])
min_fix_cus_num = float(final.agg({"fix_cus_num": "min"}).collect()[0][0])

max_avg_income_percentage = float(final.agg({"avg_income_percentage": "max"}).collect()[0][0])
min_avg_income_percentage = float(final.agg({"avg_income_percentage": "min"}).collect()[0][0])

max_avg_age_percentage = float(final.agg({"avg_age_percentage": "max"}).collect()[0][0])
min_avg_age_percentage = float(final.agg({"avg_age_percentage": "min"}).collect()[0][0])

max_count_of_bigorder = float(final.agg({"count_of_bigorder": "max"}).collect()[0][0])
min_count_of_bigorder = float(final.agg({"count_of_bigorder": "min"}).collect()[0][0])

max_fix_cus_num = float(final.agg({"fix_cus_num": "max"}).collect()[0][0])
min_fix_cus_num = float(final.agg({"fix_cus_num": "min"}).collect()[0][0])


In [4]:
final_1 = final.withColumn('Take_rate', (final.Take_rate - min_Take_rate)*1000/(max_Take_rate-min_Take_rate))\
    .withColumn('Avg_amount_monthly', (final.Avg_amount_monthly - min_Avg_amount_monthly)*1000/(max_Avg_amount_monthly - min_Avg_amount_monthly))\
    .withColumn('Avg_count_monthly', (final.Avg_count_monthly - min_Avg_count_monthly)*1000/(max_Avg_count_monthly - min_Avg_count_monthly))\
    .withColumn('Order_avg_value', (final.Order_avg_value - min_Order_avg_value)*1000/(max_Order_avg_value - min_Order_avg_value))\
    .withColumn('avg_prob_fraud_cus', -(final.avg_prob_fraud_cus - min_avg_prob_fraud_cus)*1000/(max_avg_prob_fraud_cus - min_avg_prob_fraud_cus))\
    .withColumn('num_of_fraud', -(final.num_of_fraud - min_num_of_fraud)*1000/(max_num_of_fraud - min_num_of_fraud))\
    .withColumn('count_cus_per_mon', (final.count_cus_per_mon - min_count_cus_per_mon)*1000/(max_count_cus_per_mon - min_count_cus_per_mon))\
    .withColumn('avg_income_percentage', (final.avg_income_percentage - min_avg_income_percentage)*1000/(max_avg_income_percentage - min_avg_income_percentage))\
    .withColumn('avg_age_percentage', (final.avg_age_percentage - min_avg_age_percentage)*1000/(max_avg_age_percentage - min_avg_age_percentage))\
    .withColumn('count_of_bigorder', (final.count_of_bigorder - min_count_of_bigorder)*1000/(max_count_of_bigorder - min_count_of_bigorder))\
    .withColumn('fix_cus_num', (final.fix_cus_num - min_fix_cus_num)*1000/(max_fix_cus_num - min_fix_cus_num))

In [5]:
final_1 = final_1.drop('postcode')


In [9]:
final_df = final_1.withColumn('score', final_1.Take_rate + final_1.count_of_bigorder +\
                            final_1.Avg_amount_monthly + final_1.Avg_count_monthly +\
                            final_1.Order_avg_value + final_1.avg_prob_fraud_cus + final_1.num_of_fraud +\
                            final_1.count_cus_per_mon + final_1.fix_cus_num +\
                            final_1.avg_income_percentage + final_1.avg_age_percentage)

In [21]:
final_df = final_df.sort(final_df.score.desc())

In [22]:
mean_Take_rate = float(final.agg({"Take_rate": "mean"}).collect()[0][0])
sd_Take_rate = float(final.agg({"Take_rate": "stddev_pop"}).collect()[0][0])

mean_Avg_amount_monthly= float(final.agg({"Avg_amount_monthly": "mean"}).collect()[0][0])
sd_Avg_amount_monthly = float(final.agg({"Avg_amount_monthly": "stddev_pop"}).collect()[0][0])

mean_Avg_count_monthly= float(final.agg({"Avg_count_monthly": "mean"}).collect()[0][0])
sd_Avg_count_monthly = float(final.agg({"Avg_count_monthly": "stddev_pop"}).collect()[0][0])

mean_Order_avg_value = float(final.agg({"Order_avg_value": "mean"}).collect()[0][0])
sd_Order_avg_value = float(final.agg({"Order_avg_value": "stddev_pop"}).collect()[0][0])

mean_avg_prob_fraud_cus = float(final.agg({"avg_prob_fraud_cus": "mean"}).collect()[0][0])
sd_avg_prob_fraud_cus = float(final.agg({"avg_prob_fraud_cus": "stddev_pop"}).collect()[0][0])

mean_num_of_fraud = float(final.agg({"num_of_fraud": "mean"}).collect()[0][0])
sd_num_of_fraud = float(final.agg({"num_of_fraud": "stddev_pop"}).collect()[0][0])

mean_count_cus_per_mon = float(final.agg({"count_cus_per_mon": "mean"}).collect()[0][0])
sd_count_cus_per_mon = float(final.agg({"count_cus_per_mon": "stddev_pop"}).collect()[0][0])

mean_fix_cus_num = float(final.agg({"fix_cus_num": "mean"}).collect()[0][0])
sd_fix_cus_num = float(final.agg({"fix_cus_num": "stddev_pop"}).collect()[0][0])

mean_avg_income_percentage = float(final.agg({"avg_income_percentage": "mean"}).collect()[0][0])
sd_avg_income_percentage = float(final.agg({"avg_income_percentage": "stddev_pop"}).collect()[0][0])

mean_avg_age_percentage = float(final.agg({"avg_age_percentage": "mean"}).collect()[0][0])
sd_avg_age_percentage = float(final.agg({"avg_age_percentage": "stddev_pop"}).collect()[0][0])

mean_count_of_bigorder = float(final.agg({"count_of_bigorder": "mean"}).collect()[0][0])
sd_count_of_bigorder = float(final.agg({"count_of_bigorder": "stddev_pop"}).collect()[0][0])

mean_fix_cus_num = float(final.agg({"fix_cus_num": "mean"}).collect()[0][0])
sd_fix_cus_num = float(final.agg({"fix_cus_num": "stddev_pop"}).collect()[0][0])


In [27]:
final_2 = final.withColumn('Take_rate', (final.Take_rate - mean_Take_rate)/(sd_Take_rate))\
    .withColumn('Avg_amount_monthly', (final.Avg_amount_monthly - mean_Avg_amount_monthly)/(sd_Avg_amount_monthly))\
    .withColumn('Avg_count_monthly', (final.Avg_count_monthly - mean_Avg_count_monthly)/(sd_Avg_count_monthly))\
    .withColumn('Order_avg_value', (final.Order_avg_value - mean_Order_avg_value)/(sd_Order_avg_value))\
    .withColumn('avg_prob_fraud_cus', -(final.avg_prob_fraud_cus - mean_avg_prob_fraud_cus)/(sd_avg_prob_fraud_cus))\
    .withColumn('num_of_fraud', -(final.num_of_fraud - mean_num_of_fraud)/(sd_num_of_fraud))\
    .withColumn('count_cus_per_mon', (final.count_cus_per_mon - mean_count_cus_per_mon)/(sd_count_cus_per_mon))\
    .withColumn('avg_income_percentage', (final.avg_income_percentage - mean_avg_income_percentage)/(sd_avg_income_percentage))\
    .withColumn('avg_age_percentage', (final.avg_age_percentage - mean_avg_age_percentage)/(sd_avg_age_percentage))\
    .withColumn('count_of_bigorder', (final.count_of_bigorder - mean_count_of_bigorder)/(sd_count_of_bigorder))\
    .withColumn('fix_cus_num', (final.fix_cus_num - mean_fix_cus_num)/(sd_fix_cus_num))

In [28]:
final_2 = final_2.drop('postcode')

In [29]:
final_df2 = final_2.withColumn('score', final_2.Take_rate + final_2.count_of_bigorder +\
                            final_2.Avg_amount_monthly + final_2.Avg_count_monthly +\
                            final_2.Order_avg_value + final_2.avg_prob_fraud_cus + final_2.num_of_fraud +\
                            final_2.count_cus_per_mon + final_2.fix_cus_num +\
                            final_2.avg_income_percentage + final_2.avg_age_percentage)

In [31]:
final_df2 = final_df2.sort(final_df2.score.desc())

In [35]:
final_df.count()

4026

22/09/26 19:34:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 342810 ms exceeds timeout 120000 ms
22/09/26 19:34:22 WARN SparkContext: Killing executors is not supported by current scheduler.


In [34]:
final_df2.count()

4026